$$
\newcommand{\ped}[1]{_{\mathrm{#1}}}
\newcommand{\ap}[1]{^{\mathrm{#1}}}
\newcommand{\nvector}[1]{\mathbf{#1}}
\newcommand{\nmatrix}[1]{\mathit{#1}}
\newcommand{\unitvector}[1]{\hat{\nvector{e}}_{#1}}
\newcommand{\volume}{\mathcal{V}}
\newcommand{\average}[1]{\overline{#1}}
\newcommand{\rate}[1]{\dot{#1}}
\newcommand{\flux}[1]{{#1}''}
\newcommand{\curl}[1]{\nabla\times {#1}}
\newcommand{\curlv}[1]{\curl{\nvector{#1}}}
\newcommand{\divergent}[1]{\nabla \cdot #1}
\newcommand{\divergentv}[1]{\divergent{\nvector{#1}}}
\newcommand{\divergentpar}[1]{\divergent{\left( #1 \right)}}
\newcommand{\gradient}[1]{\nabla {#1}}
\newcommand{\gradientpar}[1]{\gradient{\left( {#1} \right)}}
\newcommand{\laplacian}[1]{\nabla^2 #1}
\newcommand{\laplacianpar}[1]{\laplacian{\left( #1 \right)}}
\newcommand{\vectornorm}[1]{\left\lVert #1 \right\rVert}
\newcommand{\diffp}[2]{\frac{\partial {#1}}{\partial {#2}}}
\newcommand{\diffps}[2]{\frac{\partial^2 {#1}}{\partial {#2}^2}}
\newcommand{\rvec}{\nvector{r}}
\newcommand{\nvh}{\nvector{H}}
\newcommand{\nvb}{\nvector{B}}
\newcommand{\nvrem}{\nvector{B}\ped{rem}}
\newcommand{\nvbrem}{\nvrem}
\newcommand{\nvm}{\nvector{M}}
\newcommand{\mur}{\mu\ped{r}}
\newcommand{\nvbremhat}{\hat{\nvector{B}}\ped{rem}}
\newcommand{\acoef}[2]{a_{{#1},\mathrm{#2}}}
\newcommand{\bcoef}[2]{b_{{#1},\mathrm{#2}}}
\newcommand{\Azexpr}[1]{A_{\mathrm{#1}\, z}}
\newcommand{\bremii}{B_{\mathrm{rem,II}}}
\newcommand{\bremiv}{B_{\mathrm{rem,IV}}}
\newcommand{\aIII}{\acoef{1}{III}}
\newcommand{\bIII}{\bcoef{1}{III}}
\newcommand{\nvbIII}{\nvector{B}\ped{III}}
\newcommand{\BIII}{B\ped{III}}
\newcommand{\diffd}[1]{\mathrm{d}\,{#1}}
$$

# TeslaMax Optimization

Teslamax model:

<img src="figures/teslamax.png" width=500>

In [1]:
from pathlib import Path
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import teslamax
from teslamax import TeslaMaxModel



mpl_params = {'text.usetex': True,
              'font.family': 'serif',
              'font.serif': 'Computer Modern',
              'text.latex.preamble': [r'\usepackage{engsymbols}',
                                      r'\usepackage{magref}',
                                      r'\usepackage{siunitx}']}
matplotlib.rcParams.update(mpl_params)


In [47]:
FIGSIZE_CM = 20
FIGSIZE_INCHES = FIGSIZE_CM / 2.54

FONTSIZE = 20

In [8]:
os.chdir('C:\\Users\\fabiofortkamp\\code\\TeslaMax\\')

In [9]:
os.getcwd()

'C:\\Users\\fabiofortkamp\\code\\TeslaMax'

In [44]:
import importlib
importlib.reload(teslamax)

<module 'teslamax' from 'C:\\Users\\fabiofortkamp\\code\\TeslaMax\\teslamax\\__init__.py'>

## Calculating functional

In [4]:
params_optimization = {"R_i": 0.015,
                "R_o": 0.070,
                "h_gap": 0.020,
                "R_s": 0.140,
                "h_fc": 0.010,
                "R_e": 2,
                "n_IV": 3,
                "phi_S_IV": 45,
                "n_II": 3,
                "phi_C_II": 15,
                "phi_S_II": 45,
                "B_rem_II_1": 1.4,
                "B_rem_II_2": 1.4,
                "B_rem_II_3": 1.4,
                "mu_r_II": 1.05,
                "B_rem_IV_1": 1.4,
                "B_rem_IV_2": 1.4,
                "B_rem_IV_3": 1.4,
                "mu_r_IV": 1.05,
              "linear_iron": 1,
              "mu_r_iron": 5e3,
             }

alpha_B_rem_0 = [15,30,45,15,45,135]

params_optimization = teslamax.expand_parameters_from_remanence_array(alpha_B_rem_0, params_optimization)

The code below calculate the individual contributions for each magnet block. For instance, the i-th element of the variable `F_II_x` is a list of points in the form $(B_x, B_y)$, calculated in a mesh in the air gap, where only the i-th segment in magnet II has a non-null remanence, with magnitude included in the above dictionary and oriented in the $x$ direction.

In [5]:
def calculate_F_operators(points, params):
    """
    Return (F_II_x, F_II_y, F_IV_x, F_IV_y), where each element is a list of the F-operators vector fields
    calculated at points.
    
    For instance, F_II_x[0] is an array of (B_x, B_y), calculated when only the first segment of magnet II
    is magnetized in the x-direction, with unit remanence
    """
    
    n_II = params['n_II']
    n_IV = params['n_IV']
        
    F_II_x = []
    F_II_y = []
    
    F_IV_x = []
    F_IV_y = []
    
    for k in range(0,n_II):
        F_II_x.append(teslamax.calculate_B_III_from_single_block(point=points, 
                                                   segment=k+1, 
                                                   magnet='II', 
                                                   magnitude=1.0, 
                                                    angle=0.0, 
                                                    params=params))
        
        
        F_II_y.append(teslamax.calculate_B_III_from_single_block(point=points, 
                                                   segment=k+1,
                                                   magnet='II', 
                                                   magnitude=1.0, 
                                                    angle=90.0, 
                                                    params=params))

    for j in range(0,n_IV):
        F_IV_x.append(teslamax.calculate_B_III_from_single_block(point=points, 
                                                   segment=j+1, 
                                                   magnet='IV', 
                                                   magnitude=1.0, 
                                                    angle=0.0, 
                                                    params=params))
        
        
        F_IV_y.append(teslamax.calculate_B_III_from_single_block(point=points, 
                                                   segment=j+1,
                                                   magnet='IV', 
                                                   magnitude=1.0, 
                                                    angle=90.0, 
                                                    params=params))
        
    return (F_II_x, F_II_y, F_IV_x, F_IV_y)

In [10]:
%%time

R_o = params_optimization["R_o"]
R_g = params_optimization["R_o"] + params_optimization["h_gap"]
points_air_gap = teslamax.generate_sector_mesh_points(1.001*R_o,0.999*R_g,0.0,np.pi)

F_II_x, F_II_y, F_IV_x, F_IV_y = calculate_F_operators(points_air_gap, params_optimization)

Wall time: 6min 33s


In [11]:
F_IV_x[0]

array([[  5.61575250e-01,   1.05151475e-03],
       [  5.60452185e-01,   5.04784509e-03],
       [  5.59355059e-01,   9.24951490e-03],
       ..., 
       [  5.55588761e-01,   3.03373059e-03],
       [  5.55288376e-01,   1.46317942e-03],
       [  5.55007843e-01,  -1.23864216e-04]])

Next we apply the principle of superposition, combining the contributions calculated above and using the actual remanence magnitude and angle as specified by the dictionary `params_optimization` and by the array `alpha_B_rem`.

In [12]:
def superposition_B_III(points, F_II_x, F_II_y, F_IV_x, F_IV_y, alpha_B_rem, params):
    """
    Return (x, y, B_x, B_y) based on the lists of F-operators and a vector of remanence angles.
    
    - 'points' is a list of (x,y) points where all of the F-operators are calculated.
    - Each element of 'F_{magnet}_{direction}' is an array of (B_x, B_y) vectors calculated at 'points', 
    considering only the effect of that segment
    - 'alpha_B_rem' is a vector of (n_II + n_IV) remanences, where the first n_II represent magnet II and
    the remaining elements represent magnet IV
    - 'params' is a dictionary of parameters
    
    """
    
    B_III = 0

    n_II = params["n_II"]
    for k in range(0,n_II):
        B_rem = params["B_rem_II_%d" %(k+1)]
        alpha = np.deg2rad(alpha_B_rem[k])
    
        B = B_rem * (np.cos(alpha) * F_II_x[k] + np.sin(alpha) * F_II_y[k])
    
        B_III = B_III + B
    
    n_IV = params["n_IV"]
    for j in range(0,n_IV):
        B_rem = params["B_rem_IV_%d" %(j+1)]
        alpha = np.deg2rad(alpha_B_rem[n_II + j])
        
        B = B_rem * (np.cos(alpha) * F_IV_x[j] + np.sin(alpha) * F_IV_y[j])
        
        B_III = B_III + B
        
    B_III_grid = np.concatenate((points, B_III), axis=1)
    
    return B_III_grid

In [55]:
def calculate_funcional(points,F_II_x, F_II_y, F_IV_x, F_IV_y, alpha_B_rem, params):
    """
    Return the funcional based on the lists of F-operators and a vector of remanence angles.
    
    - 'points' is a list of (x,y) points where all of the F-operators are calculated.
    - Each element of 'F_{magnet}_{direction}' is an array of (B_x, B_y) vectors calculated at 'points', 
    considering only the effect of that segment
    - 'alpha_B_rem' is a vector of (n_II + n_IV) remanences, where the first n_II represent magnet II and
    the remaining elements represent magnet IV
    - 'params' is a dictionary of parameters
    
    """
    
    B_III_data = superposition_B_III(points, F_II_x, F_II_y, F_IV_x, F_IV_y, alpha_B_rem, params)
    
    # the above statement will return [x,y,B_x,B_y]. We have to calculate the magnitude to pass it
    # to the magnetic profile data
    B_III_data = teslamax.calculate_magnitude(B_III_data)
    
    B_profile_data = teslamax.calculate_magnetic_profile(B_III_data,params)
    
    S = teslamax.calculate_average_high_field(B_profile_data)
    return S

In [58]:
S_0 = calculate_funcional(points_air_gap,F_II_x,F_II_y,F_IV_x, F_IV_y, alpha_B_rem_0,params_optimization)
print(S_0)

0.937901298419


In [67]:
def calculate_funcional_derivative(points,F_II_x, F_II_y, F_IV_x, F_IV_y, alpha_B_rem, i,params):
    """
    Return the derivative of the funcional in respect to the i-th element of the remanence angles vector.
    
    - 'points' is a list of (x,y) points where all of the F-operators are calculated.
    - Each element of 'F_{magnet}_{direction}' is an array of (B_x, B_y) vectors calculated at 'points', 
    considering only the effect of that segment
    - 'alpha_B_rem' is a vector of (n_II + n_IV) remanences, where the first n_II represent magnet II and
    the remaining elements represent magnet IV
    - 'i' is the element (0-based) in respect to which the derivative is being evaluated
    - 'params' is a dictionary of parameters
    
    """
    
    S = calculate_funcional(points,F_II_x, F_II_y, F_IV_x, F_IV_y, alpha_B_rem, params)
    
    alpha_B_rem_plus = alpha_B_rem.copy()
    delta = 1e-6
    alpha_B_rem_plus[i]  = alpha_B_rem_plus[i] + delta
    
    S_plus = calculate_funcional(points,F_II_x, F_II_y, F_IV_x, F_IV_y, alpha_B_rem_plus, params)
    
    dS = (S_plus - S)/delta
    
    return dS

In [68]:
dS_0 = calculate_funcional_derivative(points_air_gap,F_II_x,F_II_y,F_IV_x, F_IV_y, alpha_B_rem_0, 2, params_optimization)
print(dS_0)

-0.00154200641145
